#### 1.0. Import Required Libraries

In [0]:
%python
%pip install pymongo
import os
import json
import pymongo
import requests
from pyspark.sql.functions import *
from pyspark.sql.types import *

  Using cached pymongo-4.10.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (22 kB)
  Using cached dnspython-2.7.0-py3-none-any.whl.metadata (5.8 kB)
Using cached pymongo-4.10.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.9 MB)
Using cached dnspython-2.7.0-py3-none-any.whl (313 kB)
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


#### 2.0. Instantiate Global Variables

In [0]:
# Azure MySQL Server Connection Information ###################
jdbc_hostname = "upn2vz-mysql.mysql.database.azure.com"
jdbc_port = 3306
src_database = "mystore"

connection_properties = {
  "user" : "upn2vzatvirginia",
  "password" : "Akinolami6650!",
  "driver" : "org.mariadb.jdbc.Driver"
}


# MongoDB Atlas Connection Information ########################
atlas_cluster_name = "cluster0.mkvm3"
atlas_database_name = "myshop"
atlas_user_name = "upn2vz"
atlas_password = "3yseiHK29Bemx2qK"


# Data Files (JSON) Information ###############################
dst_database = "mystore"

base_dir = "dbfs:/FileStore/sales_dw"
database_dir = f"{base_dir}/sales_dw"
# Stream paths
stream_dir = f"{base_dir}/stream"
sales_stream_dir = f"{stream_dir}/sales"
api_stream_dir = f"{stream_dir}/products"
# Output paths
output_bronze = f"{database_dir}/bronze"
output_silver = f"{database_dir}/silver"
output_gold = f"{database_dir}/gold"

#### 3.0. Define Global Functions

In [0]:
##################################################################################################################
# Use this Function to Fetch a DataFrame from the MongoDB Atlas database server Using PyMongo.
##################################################################################################################

def get_mongo_dataframe(user_id, pwd, cluster_name, db_name, collection, conditions, projection, sort):
    '''Create a client connection to MongoDB'''
    mongo_uri = f"mongodb+srv://{user_id}:{pwd}@{cluster_name}.mongodb.net/{db_name}"
    
    client = pymongo.MongoClient(mongo_uri)

    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = client[db_name]
    if conditions and projection and sort:
        dframe = pd.DataFrame(list(db[collection].find(conditions, projection).sort(sort)))
    elif conditions and projection and not sort:
        dframe = pd.DataFrame(list(db[collection].find(conditions, projection)))
    else:
        dframe = pd.DataFrame(list(db[collection].find()))

    client.close()
    
    return dframe

##################################################################################################################
# Use this Function to Create New Collections by Uploading JSON file(s) to the MongoDB Atlas server.
##################################################################################################################
def set_mongo_collection(user_id, pwd, cluster_name, db_name, src_file_path, json_files):
    '''Create a client connection to MongoDB'''
    mongo_uri = f"mongodb+srv://{user_id}:{pwd}@{cluster_name}.mongodb.net/{db_name}"
    client = pymongo.MongoClient(mongo_uri)
    db = client[db_name]
    
    '''Read in a JSON file, and Use It to Create a New Collection'''
    for file in json_files:
        db.drop_collection(file)
        json_file = os.path.join(src_file_path, json_files[file])
        with open(json_file, 'r') as openfile:
            json_object = json.load(openfile)
            file = db[file]
            result = file.insert_many(json_object)

    client.close()
    
    return result

### Section II: Populate Dimensions by Ingesting Reference (Cold-path) Data 
#### 1.0. Fetch Reference Data From an Azure MySQL Database
##### 1.1. Create a New Databricks Metadata Database.

In [0]:
%sql
DROP DATABASE IF EXISTS mystore CASCADE;

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS mystore
COMMENT "DS-2002 Capstone Project Database"
LOCATION "dbfs:/FileStore/project_data/mystore"
WITH DBPROPERTIES (contains_pii = true, purpose = "Sales Analysis");

##### 1.2. Create a New Table that Sources Date Dimension Data from a Table in an Azure MySQL database. 

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW view_date
USING org.apache.spark.sql.jdbc
OPTIONS (
  url "jdbc:mysql://upn2vz-mysql.mysql.database.azure.com:3306/mystore?useSSL=true&requireSSL=true",
  dbtable "dim_date",
  user "upn2vzatvirginia",
  password "Akinolami6650!"
)

In [0]:
%sql
USE DATABASE mystore;

CREATE OR REPLACE TABLE mystore.dim_date
COMMENT "Date Dimension Table"
LOCATION "dbfs:/FileStore/lab_data/mystore/dim_date"
AS SELECT * FROM view_date

num_affected_rows,num_inserted_rows


In [0]:
%sql
DESCRIBE EXTENDED mystore.dim_date;

col_name,data_type,comment
date_key,int,null
full_date,date,null
date_name,char(11),null
date_name_us,char(11),null
date_name_eu,char(11),null
day_of_week,tinyint,null
day_name_of_week,char(10),null
day_of_month,tinyint,null
day_of_year,int,null
weekday_weekend,char(10),null


In [0]:
%sql
SELECT * FROM mystore.dim_date LIMIT 5

date_key,full_date,date_name,date_name_us,date_name_eu,day_of_week,day_name_of_week,day_of_month,day_of_year,weekday_weekend,week_of_year,month_name,month_of_year,is_last_day_of_month,calendar_quarter,calendar_year,calendar_year_month,calendar_year_qtr,fiscal_month_of_year,fiscal_quarter,fiscal_year,fiscal_year_month,fiscal_year_qtr
20000101,2000-01-01,2000/01/01,01/01/2000,01/01/2000,7,Saturday,1,1,Weekend,52,January,1,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
20000102,2000-01-02,2000/01/02,01/02/2000,02/01/2000,1,Sunday,2,2,Weekend,52,January,1,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
20000103,2000-01-03,2000/01/03,01/03/2000,03/01/2000,2,Monday,3,3,Weekday,1,January,1,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
20000104,2000-01-04,2000/01/04,01/04/2000,04/01/2000,3,Tuesday,4,4,Weekday,1,January,1,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
20000105,2000-01-05,2000/01/05,01/05/2000,05/01/2000,4,Wednesday,5,5,Weekday,1,January,1,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3


##### 1.3. Create a New Table that Sources Product Dimension Data from an Azure MySQL database.

In [0]:
%sql
CREATE TABLE mystore.dim_sales (
    row_id INT,
    order_id VARCHAR(255),
    order_date DATE,
    ship_date DATE,
    ship_mode VARCHAR(255),
    customer_id VARCHAR(255),
    customer_name VARCHAR(255),
    segment VARCHAR(255),
    city VARCHAR(255),
    state VARCHAR(255),
    country VARCHAR(255),
    postal_code VARCHAR(255),
    market VARCHAR(255),
    region VARCHAR(255),
    product_id VARCHAR(255),
    category VARCHAR(255),
    sub_category VARCHAR(255),
    product_name VARCHAR(255),
    sales DECIMAL(10, 2),
    quantity INT,
    discount DECIMAL(10, 2),
    profit DECIMAL(10, 2),
    shipping_cost DECIMAL(10, 2),
    order_priority VARCHAR(255)
);

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import json

def upload_json_to_mysql():
    # Initialize Spark Session
    spark = SparkSession.builder \
        .appName("JSON to MySQL") \
        .getOrCreate()
    
    # Define schema matching your JSON structure
    schema = StructType([
        StructField("Row ID", StringType(), True),
        StructField("Order ID", StringType(), True),
        StructField("Order Date", StringType(), True),
        StructField("Ship Date", StringType(), True),
        StructField("Ship Mode", StringType(), True),
        StructField("Customer ID", StringType(), True),
        StructField("Customer Name", StringType(), True),
        StructField("Segment", StringType(), True),
        StructField("City", StringType(), True),
        StructField("State", StringType(), True),
        StructField("Country", StringType(), True),
        StructField("Postal Code", StringType(), True),
        StructField("Market", StringType(), True),
        StructField("Region", StringType(), True),
        StructField("Product ID", StringType(), True),
        StructField("Category", StringType(), True),
        StructField("Sub-Category", StringType(), True),
        StructField("Product Name", StringType(), True),
        StructField("Sales", DecimalType(10,2), True),
        StructField("Quantity", IntegerType(), True),
        StructField("Discount", DecimalType(4,2), True),
        StructField("Profit", DecimalType(10,2), True),
        StructField("Shipping Cost", DecimalType(10,2), True),
        StructField("Order Priority", StringType(), True)
    ])

    # Read JSON file
    df = spark.read \
        .schema(schema) \
        .option("multiline", "true") \
        .json("dbfs:/FileStore/shared_uploads/upn2vz@virginia.edu/StoreSales.json send it to me by 10 that would be perfect 10 or")
    
    # MySQL connection properties
    mysql_properties = {
        "user": "upn2vzatvirginia",
        "password": "Akinolami6650!",
        "driver": "com.mysql.cj.jdbc.Driver"
    }

    # MySQL URL
    mysql_url = "jdbc:mysql://upn2vz-mysql.mysql.database.azure.com:3306/mystore?useSSL=true"
    
    # Write to MySQL
    df.write \
        .mode("overwrite") \
        .jdbc(url=mysql_url,
              table="source_sales",
              properties=mysql_properties)
    
    print("Data successfully loaded to MySQL")

upload_json_to_mysql()

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Data successfully loaded to MySQL


In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW view_sales
USING org.apache.spark.sql.jdbc
OPTIONS (
  url "jdbc:mysql://upn2vz-mysql.mysql.database.azure.com:3306/mystore?useSSL=true&requireSSL=true",
  dbtable "dim_sales",
  user "upn2vzatvirginia",
  password "Akinolami6650!"
)

com.databricks.backend.common.rpc.SparkDriverExceptions$SQLExecutionException: java.sql.SQLSyntaxErrorException: (conn=59) Table 'mystore.dim_sales' doesn't exist
	at org.mariadb.jdbc.internal.util.exceptions.ExceptionFactory.createException(ExceptionFactory.java:62)
	at org.mariadb.jdbc.internal.util.exceptions.ExceptionFactory.create(ExceptionFactory.java:158)
	at org.mariadb.jdbc.MariaDbStatement.executeExceptionEpilogue(MariaDbStatement.java:262)
	at org.mariadb.jdbc.ClientSidePreparedStatement.executeInternal(ClientSidePreparedStatement.java:229)
	at org.mariadb.jdbc.ClientSidePreparedStatement.execute(ClientSidePreparedStatement.java:149)
	at org.mariadb.jdbc.ClientSidePreparedStatement.executeQuery(ClientSidePreparedStatement.java:163)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRDD$.$anonfun$getQueryOutputSchema$2(JDBCRDD.scala:76)
	at scala.util.Using$.resource(Using.scala:269)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRDD$.$anonfun$getQueryOutputSche

In [0]:
spark.sql("""
CREATE DATABASE IF NOT EXISTS sales_dw
COMMENT "Sales Data Warehouse"
LOCATION "dbfs:/FileStore/sales_dw"
WITH DBPROPERTIES ('contains_pii' = 'true', 'purpose' = 'Sales Analysis')
""")

DataFrame[]

In [0]:
def setup_sales_stream():
    return (spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "json")
        .option("cloudFiles.schemaLocation", f"{output_bronze}/sales_schema")
        .load(sales_stream_dir)
        .createOrReplaceTempView("sales_bronze_view"))

In [0]:
def setup_product_stream():
    return (spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "json")
        .option("cloudFiles.schemaLocation", f"{output_bronze}/products_schema")
        .load(api_stream_dir)
        .createOrReplaceTempView("products_bronze_view"))

In [0]:
# 5. Create Silver Layer with Dimension Joins
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW sales_silver_view AS
SELECT 
    s.*,
    d.year,
    d.quarter,
    d.month,
    c.customer_name,
    c.segment,
    p.product_name,
    p.category
FROM sales_bronze_view s
JOIN dim_date d ON s.order_date_id = d.date_id
JOIN dim_customer c ON s.customer_id = c.customer_id
JOIN dim_product p ON s.product_id = p.product_id
""")

{"ts": "2024-12-08 14:56:49,333", "level": "ERROR", "logger": "SQLQueryContextLogger", "msg": "[TABLE_OR_VIEW_NOT_FOUND] The table or view `sales_bronze_view` cannot be found. Verify the spelling and correctness of the schema and catalog.\nIf you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.\nTo tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS. SQLSTATE: 42P01", "context": {"error_class": "TABLE_OR_VIEW_NOT_FOUND"}, "exception": {"class": "Py4JJavaError", "msg": "An error occurred while calling o383.sql.\n: org.apache.spark.sql.catalyst.ExtendedAnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `sales_bronze_view` cannot be found. Verify the spelling and correctness of the schema and catalog.\nIf you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.\nTo tolerate the error on drop u

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1915622986156474>, line 2
      1 # 5. Create Silver Layer with Dimension Joins
----> 2 spark.sql("""
      3 CREATE OR REPLACE TEMPORARY VIEW sales_silver_view AS
      4 SELECT 
      5     s.*,
      6     d.year,
      7     d.quarter,
      8     d.month,
      9     c.customer_name,
     10     c.segment,
     11     p.product_name,
     12     p.category
     13 FROM sales_bronze_view s
     14 JOIN dim_date d ON s.order_date_id = d.date_id
     15 JOIN dim_customer c ON s.customer_id = c.customer_id
     16 JOIN dim_product p ON s.product_id = p.product_id
     17 """)

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49  

In [0]:
# 6. Create Gold Layer Aggregations
spark.sql("""
CREATE OR REPLACE TABLE sales_dw.fact_sales_metrics AS
SELECT 
    d.year,
    d.quarter,
    c.segment,
    p.category,
    COUNT(DISTINCT s.order_id) as total_orders,
    SUM(s.quantity) as total_units,
    SUM(s.sales_amount) as total_sales,
    AVG(s.profit) as avg_profit,
    STDDEV(s.sales_amount) as sales_std_dev
FROM sales_silver_view s
JOIN dim_date d ON s.order_date_id = d.date_id
JOIN dim_customer c ON s.customer_id = c.customer_id
JOIN dim_product p ON s.product_id = p.product_id
GROUP BY d.year, d.quarter, c.segment, p.category
""")


{"ts": "2024-12-08 14:56:58,498", "level": "ERROR", "logger": "SQLQueryContextLogger", "msg": "[TABLE_OR_VIEW_NOT_FOUND] The table or view `sales_silver_view` cannot be found. Verify the spelling and correctness of the schema and catalog.\nIf you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.\nTo tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS. SQLSTATE: 42P01", "context": {"error_class": "TABLE_OR_VIEW_NOT_FOUND"}, "exception": {"class": "Py4JJavaError", "msg": "An error occurred while calling o383.sql.\n: org.apache.spark.sql.catalyst.ExtendedAnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `sales_silver_view` cannot be found. Verify the spelling and correctness of the schema and catalog.\nIf you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.\nTo tolerate the error on drop u

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1915622986156475>, line 2
      1 # 6. Create Gold Layer Aggregations
----> 2 spark.sql("""
      3 CREATE OR REPLACE TABLE sales_dw.fact_sales_metrics AS
      4 SELECT 
      5     d.year,
      6     d.quarter,
      7     c.segment,
      8     p.category,
      9     COUNT(DISTINCT s.order_id) as total_orders,
     10     SUM(s.quantity) as total_units,
     11     SUM(s.sales_amount) as total_sales,
     12     AVG(s.profit) as avg_profit,
     13     STDDEV(s.sales_amount) as sales_std_dev
     14 FROM sales_silver_view s
     15 JOIN dim_date d ON s.order_date_id = d.date_id
     16 JOIN dim_customer c ON s.customer_id = c.customer_id
     17 JOIN dim_product p ON s.product_id = p.product_id
     18 GROUP BY d.year, d.quarter, c.segment, p.category
     19 """)

File /databricks/spark/python/pyspark/instrumentation

In [0]:
# 7. Start Streaming Query
def start_streaming():
    return (spark.table("sales_silver_view")
        .writeStream
        .format("delta")
        .outputMode("append")
        .option("checkpointLocation", f"{output_silver}/checkpoint")
        .table("sales_dw.fact_sales"))



In [0]:
# 8. Analysis Views
spark.sql("""
CREATE OR REPLACE VIEW sales_dw.vw_customer_analysis AS
SELECT 
    customer_id,
    customer_name,
    segment,
    SUM(sales_amount) as total_sales,
    COUNT(DISTINCT order_id) as order_count,
    AVG(sales_amount) as avg_order_value,
    STDDEV(sales_amount) as order_value_std_dev
FROM sales_dw.fact_sales
GROUP BY customer_id, customer_name, segment
""")